In [30]:
import cv2
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout
from PIL import Image
from keras.preprocessing import image

In [31]:
class otherModel:
    def __init__(self,  model=None):
        self.model = model

    def set_model(self, model):
        self.model = model
        
    def train_model(self, x_train, y_train, x_test, y_test, epochs=100, batch_size=200):
        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)
        self.model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs,
                    validation_split=0.2, # Доля валидационной выборки для контроля переобучения
                    verbose=2) #verbose=0 ничего не покажет, verbose=1 покажет анимированный индикатор выполнения (progress_bar), verbose=2 будет просто указать номер эпохи

    def predict_dense(self, img):
            print("shape:", img.shape)  # Добавим вывод формы входных данных
            # Получаем вероятности для каждого класса
            probabilities = self.model.predict(img)
            # Используем argmax для получения индекса класса с наибольшей вероятностью
            return np.argmax(probabilities)

    def print_metrics(self):
        history_dict = self.model.history.history
        best_epoch = np.argmax(history_dict['val_accuracy']) + 1

        print("Метрики обучения:")
        print("---------------------------------------------------------")
        print(f"Максимальная точность на тренировочном наборе: {max(history_dict['accuracy']):.4f}")
        print(f"Минимальная потеря на тренировочном наборе: {min(history_dict['loss']):.4f}")
        print(f"Максимальная точность на валидационном наборе: {max(history_dict['val_accuracy']):.4f}")
        print(f"Минимальная потеря на валидационном наборе: {min(history_dict['val_loss']):.4f}")
        print(f"Наивысшая точность достигнута на эпохе: {best_epoch}")

In [32]:
class DenseModel:
    def __init__(self):
        self.model = None
        self.build_model()  # Инициализируем модель в конструкторе
        self.other_model = otherModel(self.model)  # Добавляем экземпляр класса otherModel
    
    def build_model(self, input_shape=(28, 28), num_classes=10):
        # Создаем последовательную модель нейронной сети
        self.model = Sequential()
        # Входной полносвязный слой
        # - 800 нейронов: определяет количество нейронов в слое
        # - input_dim=784: указывает на количество входов в каждый нейрон, в данном случае, 784, так как входные данные представляют собой изображения размером 28x28 пикселей, которые были преобразованы в одномерный массив размером 784 пикселя
        # - activation="relu": функция активации ReLU (Rectified Linear Unit) применяется для добавления нелинейности в сеть
        self.model.add(Dense(800, input_dim=784, activation="relu"))
        # Выходной полносвязный слой
        # - 10 нейронов: количество нейронов в выходном слое соответствует количеству классов, в данном случае, 10 классов (цифры от 0 до 9)
        # - activation="softmax": функция активации Softmax применяется для преобразования выходов сети в вероятности принадлежности к каждому классу
        self.model.add(Dense(10, activation="softmax"))
        # Компиляция модели
        # - loss="categorical_crossentropy": функция потерь, используемая для оценки ошибки сети; в данном случае, используется категориальная перекрестная энтропия, так как у нас задача классификации с несколькими классами
        # - optimizer="SGD": оптимизатор, используемый для обучения сети; в данном случае, используется стохастический градиентный спуск (SGD)
        # - metrics=["accuracy"]: метрики, используемые для оценки производительности сети во время обучения; в данном случае, используется метрика точности (accuracy)
        self.model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])



    def train_model(self, x_train, y_train, x_test, y_test, epochs=100, batch_size=64):
        self.other_model.train_model(x_train, y_train, x_test, y_test, epochs, batch_size)  # Вызываем метод из класса otherModel

    def print_metrics(self):
        self.other_model.print_metrics()   # Вызываем метод из класса otherModel
    
    def transformation_img(self, img_path):
        img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
        # Преобразуем картинку в массив
        x = image.img_to_array(img)
        x = x.reshape(1,784)
        # Нормализуем изображение
        x = 255 - x
        x /= 255
        # Выпрямляем изображение в одномерный массив
        x = x.flatten()
        # Добавляем размерность батча (1, 784)
        x = np.expand_dims(x, axis=0)
        return x
    
    def predict_dense(self, img):
        print("Предсказанное число ", self.other_model.predict_dense(self.transformation_img(img)))
        


In [33]:
class ConvModel:
    def __init__(self):
        self.model = None
        self.build_model()  # Инициализируем модель в конструкторе
        self.other_model = otherModel(self.model)  # Добавляем экземпляр класса otherModel

    def build_model(self, input_shape=(28, 28, 1), num_classes=10):
        # Создаем последовательную модель нейронной сети
        self.model = Sequential()
        # Первый сверточный слой
        # 28 - количество фильтров, (3, 3) - размер ядра свертки, padding='same' - добавляет нулевые значения входам, чтобы размер выхода был таким же, как и у входа
        # input_shape=(28, 28, 1) - размерность входных данных (ширина, высота, количество каналов)
        # activation='relu' - функция активации ReLU (Rectified Linear Unit)
        self.model.add(Conv2D(28, (3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
        # Второй сверточный слой
        # Аналогично первому слою, но без указания input_shape, так как это уже продолжение предыдущего слоя
        self.model.add(Conv2D(28, (3, 3), activation='relu', padding='same'))
        # Первый слой подвыборки
        # pool_size=(2, 2) - размер окна для уменьшения размерности
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        # Слой регуляризации Dropout
        # 0.25 - доля входных единиц, которые будут исключены случайным образом на каждом обновлении во время обучения для предотвращения переобучения
        self.model.add(Dropout(0.25))
        # Третий сверточный слой
        self.model.add(Conv2D(2 * 28, (3, 3), padding='same', activation='relu'))
        # Четвертый сверточный слой
        self.model.add(Conv2D(2 * 28, (3, 3), activation='relu'))
        # Второй слой подвыборки
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        # Слой регуляризации Dropout
        self.model.add(Dropout(0.25))
        # Слой преобразования данных из 2D представления в плоское
        self.model.add(Flatten())
        # Полносвязный слой для классификации
        # 8 * 28 - количество нейронов в слое
        self.model.add(Dense(8 * 28, activation='relu'))
        # Слой регуляризации Dropout
        self.model.add(Dropout(0.5))
        # Выходной полносвязный слой
        # 10 - количество классов для классификации
        self.model.add(Dense(10, activation='softmax'))
        # Компилируем модель
        # loss='categorical_crossentropy' - функция потерь для многоклассовой классификации
        # optimizer='adam' - оптимизатор Adam
        # metrics=['accuracy'] - метрика, используемая для оценки производительности модели
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


    def transformation_img(self, img_path):
        img = image.load_img(img_path, target_size=(28, 28), color_mode="grayscale")
        # Преобразуем картинку в массив
        x = image.img_to_array(img)
        x= 255 - x
        # Нормализуем изображение
        x /= 255
        # для второй нейронки
        x = np.expand_dims(x, axis=0)
        return x
    
    def train_model(self, x_train, y_train, x_test, y_test, epochs=15, batch_size=64):
        self.other_model.train_model(x_train, y_train, x_test, y_test, epochs, batch_size)  # Вызываем метод из класса otherModel

    def print_metrics(self):
        self.other_model.print_metrics()   # Вызываем метод из класса otherModel
    
    def predict_dense(self, img):
        print("Предсказанное число (Dense Model):", self.other_model.predict_dense(self.transformation_img(img)))

In [34]:
# Загрузка данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Нормализация изображений
x_train = x_train / 255.0
x_test = x_test / 255.0

In [35]:
# Создание экземпляров классов моделей
dense_model = DenseModel()

In [36]:
conv_model = ConvModel()

In [37]:
# Обучение полносвязной нейронной сети
dense_model.build_model()

# Преобразуйте форму ваших входных данных
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

dense_model.train_model(x_train_flat, y_train, x_test_flat, y_test)

Epoch 1/100
750/750 - 8s - 10ms/step - accuracy: 0.8072 - loss: 0.8682 - val_accuracy: 0.8888 - val_loss: 0.4562
Epoch 2/100
750/750 - 5s - 7ms/step - accuracy: 0.8901 - loss: 0.4245 - val_accuracy: 0.9076 - val_loss: 0.3546
Epoch 3/100
750/750 - 5s - 7ms/step - accuracy: 0.9027 - loss: 0.3561 - val_accuracy: 0.9139 - val_loss: 0.3145
Epoch 4/100
750/750 - 5s - 7ms/step - accuracy: 0.9109 - loss: 0.3219 - val_accuracy: 0.9206 - val_loss: 0.2911
Epoch 5/100
750/750 - 8s - 10ms/step - accuracy: 0.9171 - loss: 0.2984 - val_accuracy: 0.9254 - val_loss: 0.2733
Epoch 6/100
750/750 - 8s - 11ms/step - accuracy: 0.9220 - loss: 0.2805 - val_accuracy: 0.9277 - val_loss: 0.2598
Epoch 7/100
750/750 - 7s - 9ms/step - accuracy: 0.9264 - loss: 0.2655 - val_accuracy: 0.9315 - val_loss: 0.2477
Epoch 8/100
750/750 - 7s - 9ms/step - accuracy: 0.9303 - loss: 0.2525 - val_accuracy: 0.9341 - val_loss: 0.2377
Epoch 9/100
750/750 - 7s - 9ms/step - accuracy: 0.9335 - loss: 0.2415 - val_accuracy: 0.9358 - val_lo

In [38]:
dense_model.print_metrics()

Метрики обучения:
---------------------------------------------------------
Максимальная точность на тренировочном наборе: 0.9911
Минимальная потеря на тренировочном наборе: 0.0405
Максимальная точность на валидационном наборе: 0.9763
Минимальная потеря на валидационном наборе: 0.0821
Наивысшая точность достигнута на эпохе: 97


In [39]:
def dense(path):
    dense_model.predict_dense(path)

In [40]:
dense('number/0.jpg')
dense('number/1.jpg')
dense('number/2.jpg')
dense('number/3.jpg')
dense('number/4.jpg')

shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Предсказанное число  0
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Предсказанное число  6
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Предсказанное число  2
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Предсказанное число  3
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Предсказанное число  4


In [41]:
dense('number/5.jpg')
dense('number/6.jpg')
dense('number/7.jpg')
dense('number/8.jpg')
dense('number/9.jpg')

shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Предсказанное число  5
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Предсказанное число  7
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Предсказанное число  2
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Предсказанное число  5
shape: (1, 784)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Предсказанное число  3


In [42]:
# Изменение формы входных данных для сверточной сети
x_train_conv = x_train.reshape(-1, 28, 28, 1)
x_test_conv = x_test.reshape(-1, 28, 28, 1)

# Обучение сверточной нейронной сети
conv_model.build_model()

In [43]:
conv_model.train_model(x_train_conv, y_train, x_test_conv, y_test)

Epoch 1/15
750/750 - 64s - 85ms/step - accuracy: 0.9251 - loss: 0.2361 - val_accuracy: 0.9847 - val_loss: 0.0545
Epoch 2/15
750/750 - 68s - 91ms/step - accuracy: 0.9757 - loss: 0.0758 - val_accuracy: 0.9875 - val_loss: 0.0432
Epoch 3/15
750/750 - 66s - 88ms/step - accuracy: 0.9822 - loss: 0.0571 - val_accuracy: 0.9905 - val_loss: 0.0346
Epoch 4/15
750/750 - 62s - 83ms/step - accuracy: 0.9857 - loss: 0.0471 - val_accuracy: 0.9923 - val_loss: 0.0280
Epoch 5/15
750/750 - 61s - 81ms/step - accuracy: 0.9880 - loss: 0.0394 - val_accuracy: 0.9918 - val_loss: 0.0312
Epoch 6/15
750/750 - 63s - 84ms/step - accuracy: 0.9881 - loss: 0.0374 - val_accuracy: 0.9918 - val_loss: 0.0292
Epoch 7/15
750/750 - 62s - 82ms/step - accuracy: 0.9896 - loss: 0.0330 - val_accuracy: 0.9933 - val_loss: 0.0257
Epoch 8/15
750/750 - 71s - 95ms/step - accuracy: 0.9910 - loss: 0.0267 - val_accuracy: 0.9933 - val_loss: 0.0259
Epoch 9/15
750/750 - 63s - 84ms/step - accuracy: 0.9915 - loss: 0.0274 - val_accuracy: 0.9935 - 

In [44]:
conv_model.print_metrics()

Метрики обучения:
---------------------------------------------------------
Максимальная точность на тренировочном наборе: 0.9940
Минимальная потеря на тренировочном наборе: 0.0182
Максимальная точность на валидационном наборе: 0.9939
Минимальная потеря на валидационном наборе: 0.0237
Наивысшая точность достигнута на эпохе: 12


In [45]:
def conv(path):
    conv_model.predict_dense(path)

In [46]:
img = cv2.imread('number/8.jpg')

In [49]:
conv('number/0.jpg')
conv('number/1.jpg')
conv('number/2.jpg')
conv('number/3.jpg')
conv('number/4.jpg')

shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Предсказанное число (Dense Model): 0
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Предсказанное число (Dense Model): 6
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Предсказанное число (Dense Model): 2
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Предсказанное число (Dense Model): 3
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Предсказанное число (Dense Model): 4


In [50]:
conv('number/5.jpg')
conv('number/6.jpg')
conv('number/7.jpg')
conv('number/8.jpg')
conv('number/9.jpg')


shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Предсказанное число (Dense Model): 5
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Предсказанное число (Dense Model): 5
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Предсказанное число (Dense Model): 7
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Предсказанное число (Dense Model): 8
shape: (1, 28, 28, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Предсказанное число (Dense Model): 9
